# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np

# Load dataset and filter

In [2]:
cwd = os.getcwd()

simp_path4 = 'raw_data/df.csv'
abs_path4 = os.path.abspath(os.path.join(cwd,'..',simp_path4))
df_500 = pd.read_csv(abs_path4)
df_500.date = pd.to_datetime(df_500.date)

In [ ]:
df_500['year'] = pd.DatetimeIndex(df_500['date']).year
df_500['month'] = pd.DatetimeIndex(df_500['date']).month
df_500['day'] = pd.DatetimeIndex(df_500['date']).day

In [ ]:
df_500 = df_500[df_500.year>1999]

In [ ]:
list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']

In [ ]:
df_top10 = df_500[df_500['ticker'].isin(list_of_10_stocks)]

In [ ]:
df_top10

In [ ]:
df_T = df_top10[df_top10['ticker'] == "T"]

In [ ]:
len(df_T)

# Create Sequences for LSTM

-filter only 14 years

In [ ]:
df_top10_14y = df_top10[df_top10.year<2015]

In [ ]:
df_top10_14y['ticker'].unique()

In [ ]:
df_T = df_top10_14y[df_top10_14y['ticker'] == "T"]

In [ ]:
len(df_T)

-each day

In [ ]:
df_T

In [ ]:
stock = 'T'
sequence_a = []
y = []
for i in range(0, 3600, 36):
    day = []
    df_ = df_top10_14y[df_top10_14y['ticker'] == stock]
    df_ = df_.iloc[i:i+37,3:6]
    for p in range(36):
        day.append(df_.iloc[p,:])       
    sequence_a.append(day)
    y.append(df_.iloc[36,0])

In [ ]:
len(y)

In [ ]:
X = np.array(sequence_a).astype(np.float32)

In [ ]:
X.shape

In [ ]:
X[2]

In [ ]:
y[2]

In [ ]:
df_T.iloc[107:110,:]

In [ ]:
import numpy as np

# --- SEQUENCE A (AMZN)

day_1 = [10, 25, 50]  # OBSERVATION 1 [adj_close, low, high]
day_2 = [13, 10, 70]  # OBSERVATION 2 [adj_close, low, high]
day_3 = [ 9,  5, 90]
day_4 = [ 7,  0, 95]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = 110 # adj_close of +1, +7, +21, +63

# --- SEQUENCE B (T)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = 125

# --- SEQUENCE C (AAPL)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = 30

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.array([y_a, y_b, y_c]).astype(np.float32)

print(X.shape)
print(y.shape)

In [ ]:
from stock_prices_predictions.get_data import get_lstm_training_data

X, y = get_lstm_training_data('AAPL', 1)

In [ ]:
print(y)

In [ ]:
X[0]

In [ ]:
X.shape

# Create test data

In [3]:
def get_lstm_test_data(stock, prediction_timeframe):
    '''returns X and y for the training of the LSTM'''
    cwd = os.getcwd()
    simp_path4 = 'raw_data/df.csv'
    abs_path4 = os.path.abspath(os.path.join(cwd,'..',simp_path4))
    df_500 = pd.read_csv(abs_path4)
    list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']
    df_top500 = df_500[df_500['ticker'].isin(list_of_10_stocks)]
    df_500.date = pd.to_datetime(df_500.date)
    df_500['year'] = pd.DatetimeIndex(df_500['date']).year
    df_500['month'] = pd.DatetimeIndex(df_500['date']).month
    df_500['day'] = pd.DatetimeIndex(df_500['date']).day
    df_top10_3y = df_500[df_500.year>2015]

    sequence_a = []
    y = []
    for i in range(0, 900, 36):
        day = []
        df_ = df_top10_3y[df_top10_3y['ticker'] == stock]
        df_ = df_.iloc[i:i+36+prediction_timeframe,3:4]
        for p in range(36):
            day.append(df_.iloc[p,:]) 
        sequence_a.append(day)
        X = np.array(sequence_a).astype(np.float32)
        y.append(np.array(df_.iloc[35+prediction_timeframe,0]))

    return (X, np.array(y))

In [4]:
get_lstm_test_data('T', 1)

IndexError: single positional indexer is out-of-bounds